In [ ]:
import pandas as pd
import altair as alt
import sqlite3
import os, json
import evaluate, meta, modeling, data_wrangling

In [ ]:
code_name = 'high_stress_long_pretraining'
tf_root = '/home/jupyter/tf'

cfg = meta.ModelConfig.from_json(os.path.join(tf_root, "models", code_name, "model_config.json"))
model = modeling.HS04Model(cfg)
data = data_wrangling.MyData()
cfg.batch_name = None
oral_test = evaluate.EvalOral(cfg, model, data)

In [ ]:
oral_test.eval('strain')
meta.csv_to_bigquery(os.path.join(cfg.path['model_folder'], 'eval', 'strain_item_df.csv'), dataset_name="triangle_oral", table_name="strain")

In [ ]:
oral_test.eval('taraban')
meta.csv_to_bigquery(os.path.join(cfg.path['model_folder'], 'eval', 'taraban_item_df.csv'), dataset_name="triangle_oral", table_name="taraban")